In [1]:
import json
import os
from pyquaternion import Quaternion
import pandas as pd
import pickle
import numpy as np

In [2]:
data_dir = '/work/apperception/data/nuScenes/full-dataset-v1.0/Mini/'

In [3]:
full_data_dir = '/work/apperception/data/nuScenes/full-dataset-v1.0/Trainval/'

In [4]:
base_dir = full_data_dir
folder = 'v1.0-trainval'

# base_dir = data_dir
# folder = 'v1.0-mini'

In [5]:
with open(os.path.join(base_dir, folder, 'calibrated_sensor.json')) as f:
    calibrated_sensor_json = json.load(f)

with open(os.path.join(base_dir, folder, 'category.json')) as f:
    category_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample.json')) as f:
    sample_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample_data.json')) as f:
    sample_data_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample_annotation.json')) as f:
    sample_annotation_json = json.load(f)

with open(os.path.join(base_dir, folder, 'instance.json')) as f:
    instance_json = json.load(f)

with open(os.path.join(base_dir, folder, 'scene.json')) as f:
    scene_json = json.load(f)

with open(os.path.join(base_dir, folder, 'ego_pose.json')) as f:
    ego_pose_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sensor.json')) as f:
    sensor_json = json.load(f)

In [6]:
# df_sample_data = pd.DataFrame(sample_data_json)

In [7]:
# df_sample = pd.DataFrame(sample_json)

In [8]:
# df_ego_pose = pd.DataFrame(ego_pose_json)

In [9]:
# df_scene = pd.DataFrame(scene_json)

In [10]:
# df_sample_annotation = pd.DataFrame(sample_annotation_json)

In [11]:
# df_instance = pd.DataFrame(instance_json)

In [12]:
# df_category = pd.DataFrame(category_json)

In [13]:
files = os.listdir(os.path.join(full_data_dir, 'experiment_data'))

In [14]:
# sample_data_final = pd.DataFrame(columns=['token', 'sample_token', 'timestamp', 'filename', 'camera_translation', 'camera_rotation', 'camera_intrinsic', 'ego_rotation', 'ego_translation', 'scene_name', 'ego_heading', 'camera_heading'])
# sample_annotation_final = pd.DataFrame(columns=['token', 'sample_token', 'instance_token', 'translation', 'size', 'rotation', 'category', 'heading'])

In [15]:
import math

def normalizeAngle(angle) -> float:
    while angle > math.pi:
        angle -= math.tau
    while angle < -math.pi:
        angle += math.tau
    assert -math.pi <= angle <= math.pi
    return angle

def get_heading(rotation):
    yaw = rotation.yaw_pitch_roll[0]
    return normalizeAngle(yaw)

def get_heading_from_north(rotation):
    yaw = rotation.yaw_pitch_roll[0]
    return normalizeAngle(yaw - (math.pi / 2))

rot = Quaternion(axis=[1, 0, 0], angle=np.pi / 2)
def get_camera_heading(rotation):
    return -get_heading(rot.rotate(rotation)) + math.pi / 2
    

In [16]:
files_set = set(files)

In [17]:
sample_data_filter = [s for s in sample_data_json if s['filename'].split('/')[2] in files_set]
# sample_data_filter = [s for s in sample_data_json if 'CAM_FRONT/' in s['filename']]

In [18]:
sample_data_filter[0]

{'token': '3a6f2bbccb76463f871c5720e7aae454',
 'sample_token': 'da3c26d4d7c44efc83d854be98efb5d7',
 'ego_pose_token': '3a6f2bbccb76463f871c5720e7aae454',
 'calibrated_sensor_token': 'bff1f6294cbc4b61a07f2d85b9b5f391',
 'timestamp': 1533153257412404,
 'fileformat': 'jpg',
 'is_key_frame': True,
 'height': 900,
 'width': 1600,
 'filename': 'samples/CAM_FRONT/n008-2018-08-01-15-52-19-0400__CAM_FRONT__1533153257412404.jpg',
 'prev': '6e250417c6c04b51b17266c9b3b12e63',
 'next': '12985a804ecb49faacb355dff0dd98eb'}

In [19]:
sample_tokens = set([s['sample_token'] for s in sample_data_filter])

In [20]:
ego_pose_tokens = set([s['ego_pose_token'] for s in sample_data_filter])

In [21]:
calibrated_sensor_tokens = set([s['calibrated_sensor_token'] for s in sample_data_filter])

In [22]:
sample_filter = [
    {
        'sample_token': s['token'],
        'scene_token': s['scene_token']
    }
    for s in sample_json
    if s['token'] in sample_tokens
]
len(sample_filter)

707

In [23]:
scene_tokens = set([s['scene_token'] for s in sample_filter])

In [24]:
calibrated_sensor_filter = [
    {
        'calibrated_sensor_token': c['token'],
        'camera_translation': c['translation'],
        'camera_rotation': c['rotation'],
        'camera_intrinsic': c['camera_intrinsic']
    }
    for c in calibrated_sensor_json
    if c['token'] in calibrated_sensor_tokens
]
len(calibrated_sensor_filter)

169

In [25]:
ego_pose_filter = [
    {
        'ego_pose_token': e['token'],
        'ego_translation': e['translation'],
        'ego_rotation': e['rotation']
    }
    for e in ego_pose_json
    if e['token'] in ego_pose_tokens
]
len(ego_pose_filter)

707

In [26]:
scene_filter = [
    {
        'scene_token': s['token'],
        'scene_name': s['name'],
    }
    for s in scene_json
    if s['token'] in scene_tokens
]
len(scene_filter)

169

In [27]:
sample_map = {
    s['sample_token']: s
    for s in sample_filter
}
calibrated_sensor_map = {
    c['calibrated_sensor_token']: c
    for c in calibrated_sensor_filter
}
ego_pose_map = {
    e['ego_pose_token']: e
    for e in ego_pose_filter
}
scene_map = {
    s['scene_token']: s
    for s in scene_filter
}

def s_map(s):
    sample = sample_map[s['sample_token']]
    calibrated_sensor = calibrated_sensor_map[s['calibrated_sensor_token']]
    ego_pose = ego_pose_map[s['ego_pose_token']]
    ego_heading = get_heading_from_north(Quaternion(ego_pose['ego_rotation']))
    camera_heading = get_camera_heading(Quaternion(calibrated_sensor['camera_rotation']))
    ret = {
        **s,
        **sample,
        **calibrated_sensor,
        **ego_pose,
        **scene_map[sample['scene_token']],
        'ego_heading': ego_heading * 180 / math.pi,
        'camera_heading': normalizeAngle(camera_heading + ego_heading) * 180 / math.pi
    }
    del ret['ego_pose_token']
    del ret['calibrated_sensor_token']
    del ret['fileformat']
    del ret['height']
    del ret['width']
    del ret['prev']
    del ret['next']
    del ret['scene_token']
    return ret

sample_data_res = [*map(s_map, sample_data_filter)]

len(sample_data_res)

707

In [28]:
sample_annotation_filter = [
    sa
    for sa in sample_annotation_json
    if sa['sample_token'] in sample_tokens
]
len(sample_annotation_filter)

21226

In [29]:
instance_tokens = set([
    sa['instance_token']
    for sa in sample_annotation_filter
])

instance_filter = [
    {
        'instance_token': i['token'],
        'category_token': i['category_token']
    }
    for i in instance_json
    if i['token'] in instance_tokens
]
len(instance_filter)

6862

In [30]:
category_tokens = set([
    i['category_token']
    for i in instance_filter
])

category_filter = [
    {
        'category_token': c['token'],
        'category': c['name']
    }
    for c in category_json
    if c['token'] in category_tokens
]
len(category_filter)

23

In [31]:
instance_map = {
    i['instance_token']: i
    for i in instance_filter
}
category_map = {
    c['category_token']: c
    for c in category_filter
}

def sa_map(sa):
    instance = instance_map[sa['instance_token']]
    ret = {
        **sa,
        **instance,
        **category_map[instance['category_token']],
        'heading': (get_heading_from_north(Quaternion(sa['rotation']))) * 180 / math.pi
    }
    
    del ret['visibility_token']
    del ret['attribute_tokens']
    del ret['prev']
    del ret['next']
    del ret['num_lidar_pts']
    del ret['num_radar_pts']
    del ret['category_token']
    
    return ret

sample_annotation_res = [*map(sa_map, sample_annotation_filter)]
len(sample_annotation_res)

21226

In [32]:
sample_data_final = pd.DataFrame(sample_data_res)

In [33]:
sample_annotation_final = pd.DataFrame(sample_annotation_res)

In [34]:
df_sample_data_keyframe = sample_data_final[sample_data_final["is_key_frame"]][
    ["token", "sample_token"]
]
sample_annotation_final = sample_annotation_final.set_index("sample_token").join(
    df_sample_data_keyframe.set_index("sample_token"), on="sample_token", rsuffix="_sample_data"
)

In [35]:
sample_data_final = sample_data_final.sort_values(by=['scene_name', 'timestamp'])

In [36]:
sample_data_final["frame_order"] = 0
scene = ""
i = 1
for index, row in sample_data_final.iterrows():
    if row['scene_name'] != scene:
        scene = row['scene_name']
        sample_data_final.loc[index, "frame_order"] = 1
        i = 2
    else:
        sample_data_final.loc[index, "frame_order"] = i
        i += 1


In [37]:
print(len(sample_data_final))

707


In [38]:
print(len(sample_annotation_final))

21226


In [39]:
with open(os.path.join(base_dir, "sample_data_experiment.pickle"), "wb") as f:
    pickle.dump(sample_data_final, f)
with open(os.path.join(base_dir, "annotation_experiment.pickle"), "wb") as f:
    pickle.dump(sample_annotation_final, f)

In [40]:
calibrated_sensor = [
    {
        **c,
        **[s for s in sensor_json if s['token'] == c['sensor_token']][0]
    }
    for c in calibrated_sensor_json
]
calibrated_sensor = [
    c
    for c in calibrated_sensor
    if c['modality'] == 'camera'
]
rot = Quaternion(axis=[1, 0, 0], angle=np.pi / 2)
[
    normalizeAngle(get_heading(rot.rotate(Quaternion(c['rotation'])))) * 180 / math.pi
    for c in calibrated_sensor
    if c['channel'] == "CAM_FRONT"
]

[89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
 89.67427924515529,
